In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from xgboost.sklearn import XGBClassifier
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import matplotlib as plt
import itertools

In [2]:
#Read Datasets
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
y_train=train["ACTION"]
target='ACTION'
#test=test.drop(['id'],axis=1)
#len(np.unique(train["ROLE_FAMILY_DESC"]))


In [3]:
testid = pd.DataFrame(test.id)
micro = pd.DataFrame(train.ACTION)

In [4]:
ID=test['id']
test=test.drop(['id'],axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32769 entries, 0 to 32768
Data columns (total 10 columns):
ACTION              32769 non-null int64
RESOURCE            32769 non-null int64
MGR_ID              32769 non-null int64
ROLE_ROLLUP_1       32769 non-null int64
ROLE_ROLLUP_2       32769 non-null int64
ROLE_DEPTNAME       32769 non-null int64
ROLE_TITLE          32769 non-null int64
ROLE_FAMILY_DESC    32769 non-null int64
ROLE_FAMILY         32769 non-null int64
ROLE_CODE           32769 non-null int64
dtypes: int64(10)
memory usage: 2.8 MB


In [5]:
target='ACTION'
predictors = [x for x in train.columns if x not in target]
predictors

['RESOURCE',
 'MGR_ID',
 'ROLE_ROLLUP_1',
 'ROLE_ROLLUP_2',
 'ROLE_DEPTNAME',
 'ROLE_TITLE',
 'ROLE_FAMILY_DESC',
 'ROLE_FAMILY',
 'ROLE_CODE']

In [6]:
rows=train.shape[0]
train=train.drop('ACTION',axis=1)
result = pd.concat([train,test])
for dfgs in predictors:
    affiliate_channel_maxs = []
    affiliate_channel_maxs_dict = {}
    affiliate_channel_maxs = list(enumerate(np.unique(result[dfgs])))
    affiliate_channel_maxs_dict = {name : i for i, name in affiliate_channel_maxs}
    result[dfgs+'_enum'] = result[dfgs].map(lambda x: affiliate_channel_maxs_dict[x]).astype(int)
new_train=result[:rows]
new_train[target]=y_train
new_test=result[rows:]
new_train['combination']=new_train['MGR_ID_enum']*10000 + new_train['RESOURCE_enum']
new_test['combination']=new_test['MGR_ID_enum']*10000 + new_test['RESOURCE_enum']
new_train.info()

/home/deepu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/deepu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32769 entries, 0 to 32768
Data columns (total 20 columns):
RESOURCE                 32769 non-null int64
MGR_ID                   32769 non-null int64
ROLE_ROLLUP_1            32769 non-null int64
ROLE_ROLLUP_2            32769 non-null int64
ROLE_DEPTNAME            32769 non-null int64
ROLE_TITLE               32769 non-null int64
ROLE_FAMILY_DESC         32769 non-null int64
ROLE_FAMILY              32769 non-null int64
ROLE_CODE                32769 non-null int64
RESOURCE_enum            32769 non-null int64
MGR_ID_enum              32769 non-null int64
ROLE_ROLLUP_1_enum       32769 non-null int64
ROLE_ROLLUP_2_enum       32769 non-null int64
ROLE_DEPTNAME_enum       32769 non-null int64
ROLE_TITLE_enum          32769 non-null int64
ROLE_FAMILY_DESC_enum    32769 non-null int64
ROLE_FAMILY_enum         32769 non-null int64
ROLE_CODE_enum           32769 non-null int64
ACTION                   32769 non-null int64
combination      

/home/deepu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
#new_test=new_test.fillna(0)
#new_test.info()
new_train.to_csv("new_train1.csv")
new_test.to_csv("new_test1.csv")

In [8]:
for i in new_train.columns:
    if i[-3:] != 'num':
        new_train = new_train.drop([i], axis=1)

In [9]:
for i in new_test.columns:
    if i[-3:] != 'num':
        new_test = new_test.drop([i], axis=1)

In [10]:
new_trains = pd.concat([new_train,micro], axis=1)
new_tests = pd.concat([new_test,testid], axis=1)

In [11]:
new_trains.to_csv("new_train.csv")
new_tests.to_csv("new_test.csv")

In [12]:
log_model = LogisticRegression(class_weight='balanced')
log_model.fit(new_train,y_train)
log_model.score(new_train,y_train)
# ID=test['id']
# test=test.drop(['id'],axis=1)

0.60279532484970555

In [13]:
# test=test.fillna(value=0)
# predicted = log_model.predict_proba(test)[:,1]
# #np.unique(predicted)
# preds = pd.DataFrame({"Id":ID,"ACTION":predicted})
# preds = preds.set_index('Id')
# preds.to_csv('logistic.csv')

In [14]:
# trains = pd.read_csv('new_train.csv')

In [15]:
# trains = trains.drop('Unnamed: 0', axis=1)

In [16]:
# extr = pd.DataFrame(trains.combination)

In [17]:
# new_train = pd.concat([new_train,extr], axis=1)

In [18]:
# affiliate_channel_maxs = []
# affiliate_channel_maxs_dict = {}
# affiliate_channel_maxs = list(enumerate(np.unique(new_train['combination'])))
# affiliate_channel_maxs_dict = {name : i for i, name in affiliate_channel_maxs}

In [19]:
# new_train['combination'+'_enum'] = new_train['combination'].map(lambda x: affiliate_channel_maxs_dict[x]).astype(int)

In [20]:
# new_train = new_train.drop(['combination'], axis=1)

In [21]:
# log_model2 = LogisticRegression(class_weight='balanced')
# log_model2.fit(new_train,y_train)
# log_model2.score(new_train,y_train)

In [22]:
# new_train = new_train.drop('combination_enum', axis=1)

In [23]:
# #1-0 Encoding
# for f in train.columns:
#     print f
#     df_all_dummy = pd.get_dummies(train[f], prefix=f)
#     train = train.drop([f], axis=1)
#     train = pd.concat((train, df_all_dummy), axis=1)
# train.head()

In [24]:
# #1-0 Encoding
# test=test.drop(['id'],axis=1)
# for f in test.columns:
#     print f
#     df_all_dummy = pd.get_dummies(test[f], prefix=f)
#     test = test.drop([f], axis=1)
#     test = pd.concat((test, df_all_dummy), axis=1)
# test.head()

In [25]:
# #Copy of data
# test_back=test
# train_back=train

In [26]:
#XGboost

# gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train, y_train)
param = {'max_depth':4, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 2
dtr=xgb.DMatrix(new_train,label=y_train)
gb = xgb.cv(params=param, dtrain=dtr, num_boost_round = num_round, nfold= 5)
gb.shape[0]
#predicted = gbm.predict(test)

2

In [27]:
gb

,test-error-mean,test-error-std,train-error-mean,train-error-std
0,0.057867,0.000780,0.057470,0.000401
1,0.057684,0.000956,0.057477,0.000417


In [28]:
from xgboost.sklearn import XGBClassifier

In [29]:
xgg = XGBClassifier(max_depth=4, silent=True, objective='binary:logistic', learning_rate=1)

In [30]:
from sklearn import cross_validation
# Compute the accuracy score for all the cross validation folds.  (much simpler than what we did before!)
scores = cross_validation.cross_val_score(xgg,new_train,y_train,cv=3)
# Take the mean of the scores (because we have one for each fold)
print(scores.mean())

0.944581713172


In [31]:
#Learning Rate
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    feat=[]
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics=["auc"] , early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob)
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    print feat_imp
    #feat.append(feat_imp)
    #return feat
    #.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')

In [32]:
# xgb1 = XGBClassifier(
#  learning_rate =0.3,
#  n_estimators=120,
#  max_depth=18,
#  min_child_weight=6,
#  gamma=0.2,
#  reg_alpha=0.2,
#  reg_lambda=0.2,
#  subsample=0.9,
#  colsample_bytree=0.8,
#  objective= 'binary:logistic',
#  #nthread=4,
#  scale_pos_weight=1,
#  seed=0)
# #modelfit(xgb1, train, predictors)
# xgb1

In [33]:
# clf = GridSearchCV(
#     xgb1,
#     {
#         #'learning_rate':[i/10.0 for i in range(1,4)],
#          #'min_child_weight': [5,6,7,8],
#          #'max_depth': [10,13,14,18,20],
#          #'n_estimators': range(15,18),
#         #'gamma':[i/10.0 for i in range(2,4)],
#         #'reg_alpha':[0.2,0.3,0.4,0.5],
#         #'reg_lambda':[0.3,0.4,0.5,0.6]
#         #'colsample_bytree': [0.6,0.7,0.8,0.85,0.9],
#     },
#     cv=5,
#     n_jobs=1,
#     verbose=5,
#     scoring='roc_auc'
# )
# clf

In [34]:
# clf.fit(train[predictors],train[target])

In [35]:
# param_test1 = { 'max_depth':range(3,7,2),'min_child_weight':range(1,4,2)}
# gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.5, n_estimators=15, max_depth=5,
#  min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
#  objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=0), 
#  param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

# gsearch1.fit(train[predictors],train[target])
# gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import randint as sp_randint
# forest = RandomForestClassifier(n_estimators=100)
forest = XGBClassifier(silent=True, objective='binary:logistic',  learning_rate=1)
# param_dist = {"max_depth": [1,9],
#               "max_features": sp_randint(1, 11),
#               "min_samples_split": sp_randint(1, 11),
#               "min_samples_leaf": sp_randint(1, 11),
#               "bootstrap": [True, False],
#               "criterion": ["gini", "entropy"]}
param_dist = {"max_depth": sp_randint(5,12),"min_child_weight": sp_randint(5,12), "n_estimators":sp_randint(20,50),  "gamma":(0.5,1), "subsample":(0.5,1)}
# run randomized search
n_iter_search = 12
random_search = RandomizedSearchCV(forest, param_distributions=param_dist,
                                   n_iter=n_iter_search)
random_search.fit(new_train, y_train)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
          fit_params={}, iid=True, n_iter=12, n_jobs=1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb977277410>, 'gamma': (0.5, 1), 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb977257510>, 'min_child_weight': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb977277390>, 'subsample': (0.5, 1)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=None, verbose=0)

In [37]:
random_search.grid_scores_

[mean: 0.94351, std: 0.00138, params: {'n_estimators': 27, 'subsample': 1, 'max_depth': 5, 'gamma': 1, 'min_child_weight': 11},
 mean: 0.94284, std: 0.00207, params: {'n_estimators': 45, 'subsample': 1, 'max_depth': 10, 'gamma': 0.5, 'min_child_weight': 6},
 mean: 0.94327, std: 0.00030, params: {'n_estimators': 32, 'subsample': 1, 'max_depth': 5, 'gamma': 0.5, 'min_child_weight': 9},
 mean: 0.94296, std: 0.00066, params: {'n_estimators': 24, 'subsample': 1, 'max_depth': 5, 'gamma': 0.5, 'min_child_weight': 11},
 mean: 0.94544, std: 0.00247, params: {'n_estimators': 20, 'subsample': 1, 'max_depth': 11, 'gamma': 0.5, 'min_child_weight': 11},
 mean: 0.94397, std: 0.00079, params: {'n_estimators': 38, 'subsample': 1, 'max_depth': 7, 'gamma': 0.5, 'min_child_weight': 10},
 mean: 0.94199, std: 0.00271, params: {'n_estimators': 34, 'subsample': 1, 'max_depth': 6, 'gamma': 0.5, 'min_child_weight': 5},
 mean: 0.94315, std: 0.00208, params: {'n_estimators': 44, 'subsample': 1, 'max_depth': 11, '

In [38]:
# print(clf.best_params_) 
# print(clf.best_score_)

In [39]:
# test.info()

In [40]:
# params = {}
# params["objective"] = "binary:logistic"
# params["n_estimators"] = "17"
# params["eval_metric"] = "auc"
# params["min_child_weight"] = 6
# params["max_depth"] = 18
# params["learning_rate"] =0.3
# params['gamma']=0.2,

# params['subsample']=0.8,
# params["colsample_bytree"]= 0.8
# params['nthread']=4,
# params['scale_pos_weight']=1,

#test=test.drop(['id'],axis=1)
#xg_dtst = xgb.DMatrix(test)
#xg_dtrn = xgb.DMatrix(train[predictors], label=y_train)
#model = xgb.train(list(params.items()), xg_dtrn)
#test.info()
#predicted = model.predict(xg_dtst)

# train=train.drop(['MGR_ID'],axis=1)
# test=test.drop(['MGR_ID'],axis=1)
# predictors.remove('MGR_ID')

# model = xgb1.fit(train[predictors], train['ACTION'], eval_metric='auc')
# predict_ac = xgb1.predict(test[predictors])
# predicted = xgb1.predict_proba(test[predictors])[:,1]


In [41]:
# # Write Submission
# preds = pd.DataFrame({"Id":ID,"ACTION":predicted})
# preds = preds.set_index('Id')
# preds.to_csv('xgboost.csv')

In [42]:
# poly = PolynomialFeatures(2)
# new_train=pd.DataFrame(poly.fit_transform(train[predictors]))
# new_train[target]=train[target]
# new_train.head()

In [44]:
# new_test=pd.DataFrame(poly.fit_transform(test))
# new_test.head()

In [44]:
#Value_Counts 2-way
num=len(train.index)
#removed1=['ROLE_TITLE','ROLE_FAMILY']
#removed2=['ROLE_ROLLUP_1','ROLE_ROLLUP_2']
way2iter=set(train.columns).difference(set(['ACTION']))

In [45]:
way2iter=set(new_train.columns).difference(set(['ACTION']))

In [46]:
way2iter

{'MGR_ID_enum',
 'RESOURCE_enum',
 'ROLE_CODE_enum',
 'ROLE_DEPTNAME_enum',
 'ROLE_FAMILY_DESC_enum',
 'ROLE_FAMILY_enum',
 'ROLE_ROLLUP_1_enum',
 'ROLE_ROLLUP_2_enum',
 'ROLE_TITLE_enum'}

In [53]:
for i,j in itertools.combinations(way2iter,2):
    print i,"i",j,"j"
    break

ROLE_ROLLUP_1_enum i ROLE_FAMILY_DESC_enum j


In [54]:
for i,j in itertools.combinations(way2iter,2):
    print i,j
    #if ((i in removed1) & (j=='ROLE_CODE')) or ((i=='ROLE_CODE') & (j in removed2)):
    #    continue
    new=dict(new_train.groupby([i,j])[j].count())
    new_train[i+j]=pd.Series(zip(new_train[i],new_train[j])).map(new)
    new_test[i+j]=pd.Series(zip(new_test[i],new_test[j])).map(new)
    break
#train.head()
# way3lis=[("RESOURCE", "MGR_ID", "ROLE_ROLLUP_2"), 
# ("RESOURCE", "MGR_ID", "ROLE_DEPTNAME"),
# ("RESOURCE", "MGR_ID", "ROLE_CODE"),
# ("RESOURCE", "MGR_ID", "ROLE_FAMILY_DESC"),
# ("RESOURCE", "MGR_ID", "ROLE_FAMILY"),
# ("ROLE_FAMILY", "MGR_ID", "ROLE_DEPTNAME"),
# ("ROLE_FAMILY", "MGR_ID", "ROLE_ROLLUP_2"),
# ("RESOURCE", "ROLE_DEPTNAME", "ROLE_FAMILY"),
# ("RESOURCE", "ROLE_DEPTNAME", "ROLE_ROLLUP_2")]

ROLE_ROLLUP_1_enum ROLE_FAMILY_DESC_enum


In [62]:
pd.DataFrame(dict(new_train.groupby(["ROLE_ROLLUP_1_enum","ROLE_FAMILY_DESC_enum"]).count()))

MGR_ID_enum  RESOURCE_enum  \
ROLE_ROLLUP_1_enum ROLE_FAMILY_DESC_enum                               
0                  592                              1              1   
                   692                              1              1   
                   1828                             4              4   
                   2802                            10             10   
1                  4                               16             16   
                   12                              16             16   
                   35                               9              9   
                   39                               7              7   
                   43                               1              1   
                   453                              2              2   
                   455                              9              9   
                   539                              3              3   
                   847                              1              1   
                   923                              9              9   
                   1501                            10             10   
                   1527                             5              5   
                   1893                             3              3   
                   2178                             2              2   
                   2199                            13             13   
                   2543                             1              1   
                   2637                            16             16   
                   2643                             1              1   
                   2648                             1              1   
                   2664                             1              1   
                   2689                             1              1   
                   2692                            25             25   
                   2844                             1              1   
                   2855                            32             32   
                   2903                             1              1   
2                  3                                7              7   
...                                               ...            ...   
116                838                              3              3   
                   2802                             4              4   
117                2830                             1              1   
118                7                               24             24   
                   695                              5              5   
                   807                              1              1   
                   1449                             2              2   
                   2467                             2              2   
                   2676                             1              1   
                   2853                             2              2   
119                2640                             1              1   
120                2937                             1              1   
121                35                               1              1   
122                464                              4              4   
                   1109                             1              1   
123                7                                5              5   
124                614                              1              1   
126                2716                             1              1   
127                1402                             1              1   
                   1580                             7              7   
                   1688                             1              1   
                   2580                             1              1   
128                2                                5              5   
          

In [55]:
new

{(21, 2197): 93,
 (71, 1356): 13,
 (3, 170): 4,
 (21, 727): 5,
 (23, 264): 1,
 (3, 1886): 3,
 (9, 2000): 1,
 (55, 1893): 5,
 (94, 2735): 1,
 (21, 2924): 6,
 (21, 397): 1,
 (21, 568): 1,
 (21, 1267): 12,
 (21, 1710): 5,
 (34, 8): 28,
 (108, 7): 25,
 (54, 207): 2,
 (23, 1134): 1,
 (74, 1234): 2,
 (21, 510): 1,
 (14, 279): 9,
 (21, 2520): 10,
 (21, 180): 15,
 (21, 367): 7,
 (21, 794): 2,
 (21, 1493): 1,
 (11, 7): 8,
 (78, 218): 2,
 (21, 2206): 3,
 (2, 214): 2,
 (5, 2650): 1,
 (58, 2089): 5,
 (21, 720): 29,
 (21, 1163): 2,
 (21, 2033): 19,
 (27, 1404): 1,
 (47, 314): 1,
 (33, 34): 114,
 (34, 144): 2,
 (101, 2903): 1,
 (21, 2933): 3,
 (37, 779): 7,
 (21, 577): 2,
 (21, 1890): 4,
 (33, 2769): 1,
 (36, 5): 6,
 (34, 3): 37,
 (21, 263): 12,
 (21, 946): 1,
 (76, 2934): 4,
 (21, 1560): 8,
 (30, 2747): 29,
 (48, 1008): 1,
 (69, 1094): 7,
 (21, 189): 1,
 (21, 360): 2,
 (21, 803): 2,
 (73, 1029): 7,
 (15, 334): 2,
 (21, 2215): 1,
 (54, 480): 4,
 (40, 4): 7,
 (99, 1976): 8,
 (21, 729): 2,
 (21, 1359)

In [50]:
new_train

,RESOURCE_enum,MGR_ID_enum,ROLE_ROLLUP_1_enum,ROLE_ROLLUP_2_enum,ROLE_DEPTNAME_enum,ROLE_TITLE_enum,ROLE_FAMILY_DESC_enum,ROLE_FAMILY_enum,ROLE_CODE_enum,ROLE_ROLLUP_1_enumROLE_FAMILY_DESC_enum,...,ROLE_ROLLUP_2_enumROLE_CODE_enum,ROLE_ROLLUP_2_enumROLE_FAMILY_enum,ROLE_ROLLUP_2_enumMGR_ID_enum,ROLE_ROLLUP_2_enumROLE_DEPTNAME_enum,ROLE_CODE_enumROLE_FAMILY_enum,ROLE_CODE_enumMGR_ID_enum,ROLE_CODE_enumROLE_DEPTNAME_enum,ROLE_FAMILY_enumMGR_ID_enum,ROLE_FAMILY_enumROLE_DEPTNAME_enum,MGR_ID_enumROLE_DEPTNAME_enum
0,3050,4440,21,65,319,4,7,65,4,5920,...,454,1695,55,66,3583,14,22,43,62,14
1,644,162,21,69,310,34,62,67,38,12,...,19,225,10,123,81,5,5,5,39,10
2,2706,1679,50,58,14,0,2590,3,0,2,...,51,124,2,13,1256,2,184,2,295,2
3,2615,931,21,69,184,22,2357,65,23,1211,...,736,1701,62,165,4649,33,100,39,156,51
4,3616,1010,15,13,160,70,380,4,77,10,...,8,9,9,18,75,8,6,9,6,2
5,4172,1685,19,18,23,36,68,3,40,4,...,25,44,1,11,1043,1,52,2,122,1
6,1329,2051,21,69,320,60,2813,8,66,115,...,32,75,20,22,147,5,9,5,11,5
7,756,615,21,22,119,224,2605,21,234,6,...,5,5,5,14,23,6,6,6,6,6
8,1842,67,21,74,229,240,2823,1,251,190,...,40,76,30,42,201,21,7,30,7,7
9,5481,3797,33,38,13,0,2836,3,0,27,...,161,161,1,221,1256,1,528,1,848,1


In [51]:
#3way
for (i,j,k) in itertools.combinations(way2iter,3):
    print i,j,k
    #if (((i in removed1) & (j=='ROLE_CODE')) or ((i=='ROLE_CODE') & (j in removed2))) or (((j in removed1) & (k=='ROLE_CODE')) or ((j=='ROLE_CODE') & (k in removed2))):
    #    continue
    new=dict(new_train.groupby([i,j,k])[j].count())
    new_train[i+j+k]=pd.Series(zip(new_train[i],new_train[j],new_train[k])).map(new)
    new_test[i+j+k]=pd.Series(zip(new_test[i],new_test[j],new_test[k])).map(new)
#train.head()

#Value_Counts 1-way
for i in way2iter:
    counts_tot=dict(new_train[i].value_counts())
    new_train[i+'count']=new_train[i].map(counts_tot)
    new_test[i+'count']=new_test[i].map(counts_tot)
    new_test=new_test.fillna(value=0)


#train=train.drop(["ROLE_ROLLUP_1","ROLE_ROLLUP_2"],axis=1)
#test=test.drop(["ROLE_ROLLUP_1","ROLE_ROLLUP_2"],axis=1)

ROLE_ROLLUP_1_enum ROLE_FAMILY_DESC_enum RESOURCE_enum
ROLE_ROLLUP_1_enum ROLE_FAMILY_DESC_enum ROLE_TITLE_enum
ROLE_ROLLUP_1_enum ROLE_FAMILY_DESC_enum ROLE_ROLLUP_2_enum
ROLE_ROLLUP_1_enum ROLE_FAMILY_DESC_enum ROLE_CODE_enum
ROLE_ROLLUP_1_enum ROLE_FAMILY_DESC_enum ROLE_FAMILY_enum
ROLE_ROLLUP_1_enum ROLE_FAMILY_DESC_enum MGR_ID_enum
ROLE_ROLLUP_1_enum ROLE_FAMILY_DESC_enum ROLE_DEPTNAME_enum
ROLE_ROLLUP_1_enum RESOURCE_enum ROLE_TITLE_enum
ROLE_ROLLUP_1_enum RESOURCE_enum ROLE_ROLLUP_2_enum
ROLE_ROLLUP_1_enum RESOURCE_enum ROLE_CODE_enum
ROLE_ROLLUP_1_enum RESOURCE_enum ROLE_FAMILY_enum
ROLE_ROLLUP_1_enum RESOURCE_enum MGR_ID_enum
ROLE_ROLLUP_1_enum RESOURCE_enum ROLE_DEPTNAME_enum
ROLE_ROLLUP_1_enum ROLE_TITLE_enum ROLE_ROLLUP_2_enum
ROLE_ROLLUP_1_enum ROLE_TITLE_enum ROLE_CODE_enum
ROLE_ROLLUP_1_enum ROLE_TITLE_enum ROLE_FAMILY_enum
ROLE_ROLLUP_1_enum ROLE_TITLE_enum MGR_ID_enum
ROLE_ROLLUP_1_enum ROLE_TITLE_enum ROLE_DEPTNAME_enum
ROLE_ROLLUP_1_enum ROLE_ROLLUP_2_enum ROLE_CODE_

In [52]:
# train['combination']=train['MGR_ID']*10000 + train['RESOURCE']
# test['combination']=test['MGR_ID']*10000 + test['RESOURCE']

In [53]:
# train['combination2']=train['MGR_ID']*10000 + train['ROLE_FAMILY_DESC']
# test['combination2']=test['MGR_ID']*10000 + test['ROLE_FAMILY_DESC']

In [54]:
# train.shape

In [55]:
# new[(56723, 75078)]

In [56]:
# train['MGR_ID'][0]

In [57]:
# zip(train['MGR_ID'],train['RESOURCE'])

In [58]:
# from __future__ import division
# from collections import defaultdict
# from glob import glob
# import sys
# import math
# import os

# glob_files = os.getcwd()+'/En/*.csv'
# print glob(glob_files)
# loc_outfile = 'out.csv'

# def kaggle_bag(glob_files, loc_outfile, weights=[], method="average"):
#   if method == "average":
#     scores = defaultdict(float)
#   with open(loc_outfile,"wb") as outfile:
#     for i, glob_file in enumerate( glob(glob_files) ):
#       print "parsing:", glob_file
#       # sort glob_file by first column, ignoring the first line
#       lines = open(glob_file).readlines()
#       lines = [lines[0]] + sorted(lines[1:])
#       for e, line in enumerate( lines ):
#         if i == 0 and e == 0:
#           outfile.write(line)
#         if e > 0:
#           row = line.strip().split(",")
#           if scores[(e,row[0])] == 0:
#             scores[(e,row[0])] = 1
#           scores[(e,row[0])] *= float(row[1])
#     for j,k in sorted(scores):
#       outfile.write("%s,%f\n"%(k,math.pow(scores[(j,k)],1/(i+1))))
#     print("wrote to %s"%loc_outfile)

# kaggle_bag(glob_files, loc_outfile)

In [59]:
# from __future__ import division
# from collections import defaultdict
# from glob import glob
# import sys
# import math
# import os

# def kaggle_bag(array, loc_outfile, weights=[0.8,0.2],method="average"):
#   if method == "average":
#     scores = defaultdict(float)
#   with open(loc_outfile,"wb") as outfile:
#     for i, glob_file in enumerate(array):
#       print "parsing:", i,glob_file
#       # sort glob_file by first column, ignoring the first line
#       lines = open(glob_file).readlines()
#       lines = [lines[0]] + sorted(lines[1:])
#       for e, line in enumerate( lines ):
#         if i == 0 and e == 0:
#           outfile.write(line)
#         if e > 0:
#           row = line.strip().split(",")
#           scores[(e,row[0])] += float(row[1])*weights[i]
#     for j,k in sorted(scores):
#       outfile.write("%s,%f\n"%(k,(scores[(j,k)])))
#     print("wrote to %s"%loc_outfile)
# array=[ os.getcwd()+'/En/final.csv', os.getcwd()+'/En/blending.csv']

# kaggle_bag(array, loc_outfile)

In [60]:
# a=2
# b=3
# str(a)+str(b)

In [61]:
# train['MGR_ID'].mean()

In [62]:
# from sklearn.decomposition import TruncatedSVD
# from sklearn.random_projection import sparse_random_matrix
# X = sparse_random_matrix(100, 100, density=0.01, random_state=42)
# svd = TruncatedSVD(n_components=5, random_state=42)
# svd.fit(X) 

In [63]:
# test.shape

In [64]:
new_train

,RESOURCE_enum,MGR_ID_enum,ROLE_ROLLUP_1_enum,ROLE_ROLLUP_2_enum,ROLE_DEPTNAME_enum,ROLE_TITLE_enum,ROLE_FAMILY_DESC_enum,ROLE_FAMILY_enum,ROLE_CODE_enum,ROLE_ROLLUP_1_enumROLE_FAMILY_DESC_enum,...,ROLE_FAMILY_enumMGR_ID_enumROLE_DEPTNAME_enum,ROLE_ROLLUP_1_enumcount,ROLE_FAMILY_DESC_enumcount,RESOURCE_enumcount,ROLE_TITLE_enumcount,ROLE_ROLLUP_2_enumcount,ROLE_CODE_enumcount,ROLE_FAMILY_enumcount,MGR_ID_enumcount,ROLE_DEPTNAME_enumcount
0,3050,4440,21,65,319,4,7,65,4,5920,...,8,21407,6896,3,3583,4424,3583,10980,55,72
1,644,162,21,69,310,34,62,67,38,12,...,5,21407,12,30,81,3945,81,1287,10,159
2,2706,1679,50,58,14,0,2590,3,0,2,...,2,184,33,2,1256,184,1256,2636,3,546
3,2615,931,21,69,184,22,2357,65,23,1211,...,39,21407,1244,1,4649,3945,4649,10980,62,190
4,3616,1010,15,13,160,70,380,4,77,10,...,2,276,19,8,75,138,75,362,9,45
5,4172,1685,19,18,23,36,68,3,40,4,...,1,74,79,5,1043,73,1043,2636,2,179
6,1329,2051,21,69,320,60,2813,8,66,115,...,5,21407,154,409,147,3945,147,493,20,24
7,756,615,21,22,119,224,2605,21,234,6,...,6,21407,6,17,23,397,23,783,6,325
8,1842,67,21,74,229,240,2823,1,251,190,...,7,21407,225,4,201,1295,201,384,30,42
9,5481,3797,33,38,13,0,2836,3,0,27,...,1,221,27,13,1256,221,1256,2636,1,1135


In [65]:
xgst = XGBClassifier(max_depth=4, silent=True, objective='binary:logistic', learning_rate=1)
# Compute the accuracy score for all the cross validation folds.  (much simpler than what we did before!)
scores = cross_validation.cross_val_score(xgst,new_train,y_train,cv=3)
# Take the mean of the scores (because we have one for each fold)
print(scores.mean())

0.947114483125


In [66]:
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import randint as sp_randint
# forest = RandomForestClassifier(n_estimators=100)
forests = XGBClassifier(silent=True, objective='binary:logistic',  learning_rate=1)
# param_dist = {"max_depth": [1,9],
#               "max_features": sp_randint(1, 11),
#               "min_samples_split": sp_randint(1, 11),
#               "min_samples_leaf": sp_randint(1, 11),
#               "bootstrap": [True, False],
#               "criterion": ["gini", "entropy"]}
param_dist = {"max_depth": sp_randint(5,12),"min_child_weight": sp_randint(5,12), "n_estimators":sp_randint(20,50),  "gamma":(0.5,1), "subsample":(0.5,1)}
# run randomized search
n_iter_search = 12
random_search = RandomizedSearchCV(forests, param_distributions=param_dist,
                                   n_iter=n_iter_search)
random_search.fit(new_train, y_train)
random_search.grid_scores_

[mean: 0.93317, std: 0.00167, params: {'n_estimators': 39, 'subsample': 0.5, 'max_depth': 5, 'gamma': 1, 'min_child_weight': 5},
 mean: 0.93622, std: 0.00226, params: {'n_estimators': 26, 'subsample': 0.5, 'max_depth': 11, 'gamma': 1, 'min_child_weight': 9},
 mean: 0.93399, std: 0.00202, params: {'n_estimators': 28, 'subsample': 0.5, 'max_depth': 9, 'gamma': 1, 'min_child_weight': 6},
 mean: 0.94824, std: 0.00225, params: {'n_estimators': 47, 'subsample': 1, 'max_depth': 5, 'gamma': 1, 'min_child_weight': 6},
 mean: 0.94788, std: 0.00270, params: {'n_estimators': 26, 'subsample': 1, 'max_depth': 11, 'gamma': 0.5, 'min_child_weight': 5},
 mean: 0.94721, std: 0.00114, params: {'n_estimators': 28, 'subsample': 1, 'max_depth': 9, 'gamma': 1, 'min_child_weight': 6},
 mean: 0.93393, std: 0.00348, params: {'n_estimators': 38, 'subsample': 0.5, 'max_depth': 8, 'gamma': 0.5, 'min_child_weight': 6},
 mean: 0.94788, std: 0.00270, params: {'n_estimators': 26, 'subsample': 1, 'max_depth': 11, 'gamm

In [67]:
new_trainst = pd.concat([new_train,micro], axis=1)
new_testst = pd.concat([new_test,testid], axis=1)
new_trainst.to_csv("new_train_new.csv")
new_testst.to_csv("new_test_new.csv")